即時數字辨識系統

環境版本：Win10 python版本：3.9.9 

桌面新增資料夾(MNIST-Live-Detection-TFLite-main)

到：https://github.com/alankrantas/MNIST-Live-Detection-TFLite
，下載：1. mnist.tflite 2. mnist_tflite_live_detection.py，將其放到資料夾內

到終端機：pip3 install --upgrade numpy opencv-python

到：https://github.com/google-coral/pycoral/releases/tag/v2.0.0
，Win10 電腦使用 Python 3.9，就下載 tflite_runtime-2.5.0.post1-cp39-cp39-win_amd64.whl（cp39 代表 Python 3.9，amd64 即 x64）
樹莓派 OS Buster 用 xxx-cp37-cp37m-linux_armv7l.whl（樹莓派 2~4B 為 ARMv8 架構）
樹莓派 OS Bullseye 則用 xxx-cp39-cp39-linux_armv7l.whl，以此類推。

將上一步驟下載的檔案放到同個資料夾內，到終端機進到該資料夾(cd Desktop\MNIST-Live-Detection-TFLite-main)
，終端機：pip3 install tflite_runtime-2.5.0.post1-cp39-cp39-win_amd64.whl

最後接上webcam，終端機執行mnist_tflite_live_detection.py 即可進行即時數字辨識(按下q關閉程式)

競爭對手：
https://www.keyence.com.tw/landing/sensor/lp_vision-sensor_01123990.jsp?utm_medium=cpc&utm_content=KW-NB-CH-APP-IV-Core&aw=KW01kw0_10080242&gad=1&gclid=CjwKCAjwxr2iBhBJEiwAdXECw9JG8ed6vYps4LF3YqK-OrQpxzrOSr0Ge8N4i1AWI6PPHDH7eycVCRoCzYIQAvD_BwE


In [1]:
import cv2

In [3]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while(True):
    # 擷取影像
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # 彩色轉灰階
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顯示圖片
    cv2.imshow('live', frame)
    #cv2.imshow('live', gray)

    # 按下 q 鍵離開迴圈
    if cv2.waitKey(1) == ord('q'):
        break

# 釋放該攝影機裝置
cap.release()
cv2.destroyAllWindows()

下方程式可運行

In [1]:
TF_LITE_MODEL = './mnist.tflite'  # TF Lite model
IMG_W = 640  # video capture width
IMG_H = 480  # video capture height
IMG_BORDER = 40  # video capture border width (won't be used for detection)
DETECT_THRESHOLD = 0.7  # only display digits with 70%+ probability 
CONTOUR_COLOR = (0, 255, 255)  # digit frame color
LABEL_COLOR = (255, 255, 0)  # digit label color
LABEL_SIZE = 0.7  # digit label size (70%)
RUNTIME_ONLY = True  # use TF Lite runtime instead of Tensorflow

import cv2
import numpy as np

# load TF Lite model
if RUNTIME_ONLY:
    from tflite_runtime.interpreter import Interpreter
    interpreter = Interpreter(model_path=TF_LITE_MODEL)
else:
    import tensorflow as tf
    interpreter = tf.lite.Interpreter(model_path=TF_LITE_MODEL)

# prepare model
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# get input shape
INPUT_H, INPUT_W = input_details[0]['shape'][1:3]

# kernel for morphological closing
MORPH_KERNEL = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, IMG_W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, IMG_H)

while cap.isOpened():
    # get one frame
    success, frame = cap.read()
    
    # convert to gray
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # image thresholding (to black and white)
    _, frame_binary = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    # do morphological closing to filter out noise
    frame_binary = cv2.morphologyEx(frame_binary, cv2.MORPH_CLOSE, MORPH_KERNEL)
    
    # find contours (possible digits area) in the frame
    contours, _ = cv2.findContours(frame_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # iterate all contours
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        # if the area is overlapping the border, ignore it
        if x < IMG_BORDER or x + w > (IMG_W - 1) - IMG_BORDER or y < IMG_BORDER or y + h > (IMG_H - 1) - IMG_BORDER:
            continue
        
        # if the area is too small or too large, ignore it
        if w < INPUT_W // 2 or h < INPUT_H // 2 or w > IMG_W // 2 or h > IMG_H // 2:
            continue
        
        # get the image from the area
        img = frame_binary[y: y + h, x: x + w]
        
        # add padding to make the image square with extra border
        r = max(w, h)
        y_pad = ((w - h) // 2 if w > h else 0) + r // 5
        x_pad = ((h - w) // 2 if h > w else 0) + r // 5
        img = cv2.copyMakeBorder(img, top=y_pad, bottom=y_pad, left=x_pad, right=x_pad, borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
        
        # resize image to input size
        img = cv2.resize(img, (INPUT_W, INPUT_H), interpolation=cv2.INTER_AREA)

        # make prediction
        interpreter.set_tensor(input_details[0]['index'], np.expand_dims(img, axis=0))
        interpreter.invoke()
        predicted = interpreter.get_tensor(output_details[0]['index']).flatten()
        
        # get label and probability
        label = predicted.argmax(axis=0)
        prob = predicted[label]
        
        # ignore it if probability is below the threshold
        if prob < DETECT_THRESHOLD:
            continue
        
        # draw rectangle and text label around the image area
        cv2.rectangle(frame, (x, y), (x + w, y + h), CONTOUR_COLOR, 2)
        cv2.putText(frame, str(label), (x + w // 5, y - h // 5), cv2.FONT_HERSHEY_COMPLEX, LABEL_SIZE, LABEL_COLOR, 2)
    
    # display the frame
    cv2.imshow('MNIST Live Detection', frame)
    
    # exit video capture if user press 'q'
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


--------------------------------------------------------------------------------------------------------

STEAM教學網：
使用 Keras 搭配 NumPy 訓練手寫數字模型，再搭配 OpenCV KNN 演算方法 ( cv2.ml.KNearest_load )，即時辨識出手寫的阿拉伯數字

In [2]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras import utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()  # 載入訓練集

# 訓練集資料
x_train = x_train.reshape(x_train.shape[0],-1)  # 轉換資料形狀
x_train = x_train.astype('float32')/255         # 轉換資料型別
y_train = y_train.astype(np.float32)

# 測試集資料
x_test = x_test.reshape(x_test.shape[0],-1)     # 轉換資料形狀
x_test = x_test.astype('float32')/255           # 轉換資料型別
y_test = y_test.astype(np.float32)

knn=cv2.ml.KNearest_create()                    # 建立 KNN 訓練方法
knn.setDefaultK(5)                              # 參數設定
knn.setIsClassifier(True)

print('training...')
knn.train(x_train, cv2.ml.ROW_SAMPLE, y_train)  # 開始訓練
knn.save('mnist_knn.xml')                       # 儲存訓練模型
print('ok')

print('testing...')
test_pre = knn.predict(x_test)                  # 讀取測試集並進行辨識
test_ret = test_pre[1]
test_ret = test_ret.reshape(-1,)
test_sum = (test_ret == y_test)
acc = test_sum.mean()                           # 得到準確率
print(acc)

11501568/11490434 [==============================] - 0s 0us/step
training...
ok
testing...
0.9688


下方程式可運行

In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)                     # 啟用攝影鏡頭
print('loading...')
knn = cv2.ml.KNearest_load('mnist_knn.xml')   # 載入模型
print('start...')
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, img = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(img,(540,300))          # 改變影像尺寸，加快處理效率
    x, y, w, h = 400, 200, 60, 60            # 定義擷取數字的區域位置和大小
    img_num = img.copy()                     # 複製一個影像作為辨識使用
    img_num = img_num[y:y+h, x:x+w]          # 擷取辨識的區域

    img_num = cv2.cvtColor(img_num, cv2.COLOR_BGR2GRAY)    # 顏色轉成灰階
    # 針對白色文字，做二值化黑白轉換，轉成黑底白字
    ret, img_num = cv2.threshold(img_num, 127, 255, cv2.THRESH_BINARY_INV)
    output = cv2.cvtColor(img_num, cv2.COLOR_GRAY2BGR)     # 顏色轉成彩色
    img[0:60, 480:540] = output                            # 將轉換後的影像顯示在畫面右上角

    img_num = cv2.resize(img_num,(28,28))   # 縮小成 28x28，和訓練模型對照
    img_num = img_num.astype(np.float32)    # 轉換格式
    img_num = img_num.reshape(-1,)          # 打散成一維陣列資料，轉換成辨識使用的格式
    img_num = img_num.reshape(1,-1)
    img_num = img_num/255
    img_pre = knn.predict(img_num)          # 進行辨識
    num = str(int(img_pre[1][0][0]))        # 取得辨識結果

    text = num                              # 印出的文字內容
    org = (x,y-20)                          # 印出的文字位置
    fontFace = cv2.FONT_HERSHEY_SIMPLEX     # 印出的文字字體
    fontScale = 2                           # 印出的文字大小
    color = (0,0,255)                       # 印出的文字顏色
    thickness = 2                           # 印出的文字邊框粗細
    lineType = cv2.LINE_AA                  # 印出的文字邊框樣式
    cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType) # 印出文字

    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)  # 標記辨識的區域
    cv2.imshow('oxxostudio', img)
    if cv2.waitKey(50) == ord('q'):
        break     # 按下 q 鍵停止
cap.release()
cv2.destroyAllWindows()

loading...
start...


LCD屏幕數字識別

In [ ]:
# coding=utf-8
# 导入一些python包
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
 
# 定义每一个数字对应的字段
DIGITS_LOOKUP = {
    (1, 1, 1, 0, 1, 1, 1): 0,
    (0, 0, 1, 0, 0, 1, 0): 1,
    (1, 0, 1, 1, 1, 1, 0): 2,
    (1, 0, 1, 1, 0, 1, 1): 3,
    (0, 1, 1, 1, 0, 1, 0): 4,
    (1, 1, 0, 1, 0, 1, 1): 5,
    (1, 1, 0, 1, 1, 1, 1): 6,
    (1, 0, 1, 0, 0, 1, 0): 7,
    (1, 1, 1, 1, 1, 1, 1): 8,
    (1, 1, 1, 1, 0, 1, 1): 9
}
 
# 读取输入图片
image = cv2.imread("example.jpg")
 
# 将输入图片裁剪到固定大小
image = imutils.resize(image, height=500)
# 将输入转换为灰度图片
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 进行高斯模糊操作
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# 执行边缘检测
edged = cv2.Canny(blurred, 50, 200, 255)
cv2.imwrite('edge.png', edged)
 
# 在边缘检测map中发现轮廓
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# 根据大小对这些轮廓进行排序
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None
 
# 循环遍历所有的轮廓
for c in cnts:
    # 对轮廓进行近似
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # 如果当前的轮廓有4个顶点，我们返回这个结果，即LCD所在的位置
    if len(approx) == 4:
        displayCnt = approx
        break
 
# 应用视角变换到LCD屏幕上
warped = four_point_transform(gray, displayCnt.reshape(4, 2))
cv2.imwrite('warped.png', warped)
output = four_point_transform(image, displayCnt.reshape(4, 2))
 
# 使用阈值进行二值化
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cv2.imwrite('thresh1.png', thresh)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 5))
# 使用形态学操作进行处理
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
cv2.imwrite('thresh2.png', thresh)
 
# 在阈值图像中查找轮廓，然后初始化数字轮廓列表
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
digitCnts = []
 
# 循环遍历所有的候选区域
for c in cnts:
    # 计算轮廓的边界框
    (x, y, w, h) = cv2.boundingRect(c)
 
    # 如果当前的这个轮廓区域足够大，它一定是一个数字区域
    if w >= 15 and (h >= 30 and h <= 40):
        digitCnts.append(c)
 
# 从左到右对这些轮廓进行排序
digitCnts = contours.sort_contours(digitCnts, method="left-to-right")[0]
digits = []
 
# 循环处理每一个数字
i = 0
for c in digitCnts:
    # 获取ROI区域
    (x, y, w, h) = cv2.boundingRect(c)
    roi = thresh[y:y + h, x:x + w]
 
    # 分别计算每一段的宽度和高度
    (roiH, roiW) = roi.shape
    (dW, dH) = (int(roiW * 0.25), int(roiH * 0.15))
    dHC = int(roiH * 0.05)
 
    # 定义一个7段数码管的集合
    segments = [
        ((0, 0), (w, dH)),               # 上
        ((0, 0), (dW, h // 2)),           # 左上
        ((w - dW, 0), (w, h // 2)),          # 右上
        ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # 中间
        ((0, h // 2), (dW, h)),            # 左下
        ((w - dW, h // 2), (w, h)),          # 右下
        ((0, h - dH), (w, h))              # 下
    ]
    on = [0] * len(segments)
 
    # 循环遍历数码管中的每一段
    for (i, ((xA, yA), (xB, yB))) in enumerate(segments): # 检测分割后的ROI区域，并统计分割图中的阈值像素点
        segROI = roi[yA:yB, xA:xB]
        total = cv2.countNonZero(segROI)
        area = (xB - xA) * (yB - yA)
 
        # 如果非零区域的个数大于整个区域的一半，则认为该段是亮的
        if total / float(area) > 0.5:
            on[i]= 1
 
    # 进行数字查询并显示结果
    digit = DIGITS_LOOKUP[tuple(on)]
    digits.append(digit)
    cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 1)
    cv2.putText(output, str(digit), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
 
# 显示最终的输出结果
print(u"{}{}.{} \u00b0C".format(*digits))
cv2.imshow("Input", image)
cv2.imshow("Output", output)
cv2.waitKey(0)